In [ ]:
print("Hello, world!")

: 

In [ ]:
# --- Core imports
import pandas as pd

# --- Your project modules
from src.data import get_price_data
from src.strategies import signal_sma_crossover, signal_mean_reversion, align_next_bar
from src.engine import run_backtest
from src.metrics import summarize_performance, extract_trades, trade_stats, drawdown_series

# --- Plots
import matplotlib.pyplot as plt

# Display options
pd.set_option("display.expand_frame_repr", False)

: 

In [ ]:
TICKERS = ["AAPL", "MSFT"] 
START = "2022-01-01"
END   = "2022-12-31"

# Strategy params
SMA_FAST = 20
SMA_SLOW = 50

MR_LOOKBACK = 20
MR_ENTRY = 1.0
MR_EXIT  = 0.2

INITIAL_CASH = 100_000
FEE_BPS = 1.0
SLIP_BPS = 0.0


In [ ]:
df = get_price_data(TICKERS, START, END)

# Convenience helpers
def get_cols(symbol: str):
    return df[(symbol, "Close")], (df[(symbol, "Open")] if ("Open" in df[symbol].columns) else None)

df.head(3)

In [ ]:
# Pick one to explore first
SYM = "AAPL"
close, open_ = get_cols(SYM)

sig_sma_raw = signal_sma_crossover(close, fast=SMA_FAST, slow=SMA_SLOW)
sig_mr_raw  = signal_mean_reversion(close, lookback=MR_LOOKBACK, entry=MR_ENTRY, exit=MR_EXIT)

# Align to trade next bar (no look-ahead)
sig_sma = align_next_bar(sig_sma_raw)
sig_mr  = align_next_bar(sig_mr_raw)

sig_sma.head(10), sig_mr.head(10)

In [ ]:
bt_sma = run_backtest(close, sig_sma, open_=open_, initial_cash=INITIAL_CASH, fee_bps=FEE_BPS, slippage_bps=SLIP_BPS)
bt_mr  = run_backtest(close, sig_mr,  open_=open_, initial_cash=INITIAL_CASH, fee_bps=FEE_BPS, slippage_bps=SLIP_BPS)

sum_sma = summarize_performance(bt_sma["equity"])
sum_mr  = summarize_performance(bt_mr["equity"])

print("Summary (SMA):", sum_sma)
print("Summary (MR): ", sum_mr)

tr_sma = extract_trades(bt_sma)
tr_mr  = extract_trades(bt_mr)
print("\nTrades (SMA):")
display(tr_sma)
print("\nTrade stats (SMA):", trade_stats(tr_sma))

print("\nTrades (MR):")
display(tr_mr)
print("\nTrade stats (MR):", trade_stats(tr_mr))

In [ ]:
def plot_equity(equity, title="Equity Curve"):
    plt.figure(figsize=(10,4))
    plt.plot(equity.index, equity.values, label="Equity")
    plt.title(title); plt.ylabel("Equity ($)"); plt.xlabel("Date")
    plt.grid(alpha=0.3); plt.legend(); plt.show()

def plot_drawdown(equity, title="Drawdown"):
    dd = drawdown_series(equity)
    plt.figure(figsize=(10,3))
    plt.fill_between(dd.index, dd.values, 0, alpha=0.3)
    plt.title(title); plt.ylabel("Drawdown (fraction)"); plt.xlabel("Date")
    plt.show()

plot_equity(bt_sma["equity"], title=f"{SYM} – SMA {SMA_FAST}/{SMA_SLOW} Equity")
plot_drawdown(bt_sma["equity"], title=f"{SYM} – SMA Drawdown")

plot_equity(bt_mr["equity"],  title=f"{SYM} – Mean Reversion Equity")
plot_drawdown(bt_mr["equity"], title=f"{SYM} – Mean Reversion Drawdown")

In [ ]:
def plot_signals(close, signal, title="Price + Signals"):
    plt.figure(figsize=(10,4))
    plt.plot(close.index, close.values, label="Close")
    buys  = close.index[(signal.diff() > 0)]
    sells = close.index[(signal.diff() < 0)]
    plt.scatter(buys,  close.loc[buys],  marker="^", label="Buy")
    plt.scatter(sells, close.loc[sells], marker="v", label="Sell")
    plt.title(title); plt.legend(); plt.grid(alpha=0.3); plt.show()

plot_signals(close, sig_sma, title=f"{SYM} – SMA {SMA_FAST}/{SMA_SLOW} Signals")

In [ ]:
rows = []
for sym in TICKERS:
    c, o = get_cols(sym)
    s = align_next_bar(signal_sma_crossover(c, fast=SMA_FAST, slow=SMA_SLOW))
    bt = run_backtest(c, s, open_=o, initial_cash=INITIAL_CASH, fee_bps=FEE_BPS, slippage_bps=SLIP_BPS)
    summ = summarize_performance(bt["equity"])
    rows.append({"symbol": sym, **summ})

pd.DataFrame(rows)

Assumptions: daily data, next-bar execution (Open if available), long/flat, fees in bps, no leverage/shorting, float shares.

Limitations: no intraday fills, simple slippage model, no borrow costs, slow signals may underperform in choppy markets.

Next steps: parameter grid search, portfolio of multiple tickers, risk targeting (vol parity), walk-forward split.